In [ ]:
%load_ext autoreload
%autoreload 2
RootDIR = "/home/jw3514/Work/ASD_Circuits_CellType/"
import sys
sys.path.insert(1, RootDIR + 'src')
from ASD_Circuits import *
from venn import venn
import requests
import json

# DownLoad Allen IDs

Download expression data and discontinued Entrez IDs

http://api.brain-map.org/class_detail/Gene

http://api.brain-map.org/examples/doc/structures/download_data.py.html

http://help.brain-map.org/display/api/Example+Queries+for+Experiment+Metadata

http://api.brain-map.org/doc/DataSet.html

https://ftp.ncbi.nih.gov/gene/README

In [ ]:
url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=model::SectionDataSet," +  \
   "rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse']," + \
   "treatments[name$eq'ISH'],genes,plane_of_section," + \
   "rma::options,[tabular$eq'plane_of_sections.name+as+plane'," + \
   "'genes.acronym+as+gene_acronym'," + \
   "'genes.entrez_id+as+genes_entrez_id'," + \
   "'genes.ensembl_id+as+gene_ensembl_id'," + \
   "'genes.alias_tags+as+gene_alias_tags'," + \
   "'data_sets.id+as+section_data_set_id']," + \
   "[order$eq'plane_of_sections.name,genes.acronym,data_sets.id']&" + \
   "start_row=0&num_rows=all"
print(url)
r = requests.get(url, allow_redirects=True)
open(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_ISH_experiments.csv', 'wb').write(r.content)


In [ ]:
url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=model::Gene," +  \
   "rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse']," + \
   "[tabular$eq'acronym+as+gene_acronym'," + \
   "'gene_aliases.name+as+gene_aliases'," + \
   "'entrez_id+as+genes_entrez_id'," + \
   "[order$eq'acronym']&" + \
   "start_row=0&num_rows=all"
print(url)
r = requests.get(url, allow_redirects=True)
open(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_ISH_genes.csv', 'wb').write(r.content)


In [ ]:
url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=" + \
        "model::Gene," + \
        "rma::criteria,products[abbreviation$eq'Mouse']," + \
        "rma::options,[tabular$eq'genes.id','genes.acronym+as+gene_symbol'," + \
        "'genes.name+as+gene_name','genes.entrez_id+as+entrez_gene_id'," + \
        "'genes.homologene_id+as+homologene_group_id'," + \
        "'genes.alias_tags+as+gene_alias_tags']," + \
        "[order$eq'genes.acronym']&num_rows=all&start_row=0"
print(url)
r = requests.get(url, allow_redirects=True)
open(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_Genes.csv', 'wb').write(r.content)


# Mouse To Human Mapping

In [ ]:
AllenMouseGenes = pd.read_csv(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_ISH_experiments.csv')
Human2MouseHom = pd.read_csv(RootDIR + 'dat/HOM_MouseHumanSequence.rpt', delimiter="\t")

In [ ]:
Homo_IDs = set(Human2MouseHom["DB Class Key"].values)
print(len(Homo_IDs))

In [ ]:
Human2Mouse_Genes = {}
Mouse2Human_Genes = {}
# Key:EntrezID
# symbol: FOXP1
# mouseHomo: [(), ()]
# allen_section_data_set_id: [1,2]
for counter, ID in enumerate(Homo_IDs):
    tmp_df = Human2MouseHom[Human2MouseHom["DB Class Key"]==ID]
    #print(tmp_df)   
    hum_genes, mou_genes = [], []
    for i, row in tmp_df.iterrows():
        TaxonID = row["NCBI Taxon ID"]
        Symbol = row["Symbol"]
        Entrez = row["EntrezGene ID"]
        if  TaxonID == 9606:
            hum_genes.append((Symbol, Entrez))
        elif TaxonID == 10090:
            mou_genes.append((Symbol, Entrez))
    # Fill up Human2Mouse_Genes
    for (Symbol, Entrez) in hum_genes:
        if Entrez not in Human2Mouse_Genes:
            Human2Mouse_Genes[Entrez] = {}#mou_genes
            Human2Mouse_Genes[Entrez]["symbol"] = Symbol
            Human2Mouse_Genes[Entrez]["mouseHomo"] = mou_genes
        else:
            Human2Mouse_Genes[Entrez]["mouseHomo"].extend(mou_genes)
        
    # Fill up Mouse2Human_Genes
    for (Symbol, Entrez) in mou_genes:
        if Entrez not in Mouse2Human_Genes:
            Mouse2Human_Genes[Entrez] = {} #mou_genes
            Mouse2Human_Genes[Entrez]["symbol"] = Symbol
            Mouse2Human_Genes[Entrez]["humanHomo"] = hum_genes
            Mouse2Human_Genes[Entrez]["allen_section_data_set_id"] = []
        else:
            Mouse2Human_Genes[Entrez]["humanHomo"].extend(hum_genes)
            

In [ ]:
Mouse2Human_Genes_2 = {}
for k,v in Mouse2Human_Genes.items():
    Mouse2Human_Genes_2[v["symbol"]] = {}
    Mouse2Human_Genes_2[v["symbol"]]["Entrez"] = k
    Mouse2Human_Genes_2[v["symbol"]]["humanHomo"] = v["humanHomo"]

In [ ]:
pk.dump(Mouse2Human_Genes_2, open(RootDIR + 'dat/Mouse2Human_Symbol.pk', "wb"))
pk.dump(Mouse2Human_Genes, open(RootDIR + 'dat/Mouse2Human_Entrez.pk', "wb"))


In [ ]:
Discontinued_ID = {}
df = pd.read_csv(RootDIR + 'dat/gene_history.human.mouse.tsv', delimiter="\t")
Discontinued_ID = dict(zip(df["Discontinued_GeneID"].values, df["GeneID"].values))


In [ ]:
# Match Allen section_id to mouse genes with Entrez ID
def MatchAllen(symbol, entrez, row):
    # row: plane,gene_acronym,genes_entrez_id,gene_ensembl_id,gene_alias_tags,section_data_set_id
    #print(symbol, entrez)
    #print(row["gene_acronym"], row["genes_entrez_id"], row["gene_alias_tags"] )
    if str(row["genes_entrez_id"]) != "nan" and int(entrez) == int(row["genes_entrez_id"]):
        return row["section_data_set_id"]
    elif symbol == row["gene_acronym"]:
        return row["section_data_set_id"]
    else:
        if str(row["gene_alias_tags"]) == "nan":
            #print("haha")
            return None
        #print(str(row["gene_alias_tags"]), type(row["gene_alias_tags"]))
        for alias in row["gene_alias_tags"].split():
            alias = alias.strip()
            if alias == symbol:
                return row["section_data_set_id"]
    return None

Entrez_Failed_ID = []
for i, row in AllenMouseGenes.iterrows():
    if row["section_data_set_id"] == 68269058:
        print(row)
    if str(row["gene_acronym"]) != "nan":
        allen_symbol = row["gene_acronym"]
    else:
        allen_symbol = None
    if str(row["genes_entrez_id"]) != "nan":
        allen_entrez = int(row["genes_entrez_id"])
    else:
        allen_entrez = None
    if str(row["gene_alias_tags"]) != "nan":
        allen_alias = row["gene_alias_tags"].split()
    else:
        allen_alias = []
    allen_section_id = row["section_data_set_id"]
    try:
        potential_ID = int(Discontinued_ID[allen_entrez])
    except:
        potential_ID = -1
        pass
    if allen_entrez in Mouse2Human_Genes:
        Mouse2Human_Genes[allen_entrez]["allen_section_data_set_id"].append(allen_section_id)
    elif potential_ID in Mouse2Human_Genes:
        Mouse2Human_Genes[potential_ID]["allen_section_data_set_id"].append(allen_section_id)
    else:    
        Entrez_Failed_ID.append(allen_section_id)
 

In [ ]:
Mouse2Human_Genes[18430]

In [ ]:
68269058 in Entrez_Failed_ID

In [ ]:
Mouse2Human_Genes

In [ ]:
print(len(Entrez_Failed_ID))

In [ ]:
# Check missed by entrez ID
Mouse_MissSectionID = []
for K, V in Mouse2Human_Genes.items():
    if len(V["allen_section_data_set_id"]) == 0:
        Mouse_MissSectionID.append(K)
print(len(Mouse_MissSectionID))

In [ ]:
Unlinked_AllenMouseGenes = AllenMouseGenes[AllenMouseGenes["section_data_set_id"].isin(
    Entrez_Failed_ID)]
Symbol_Failed_ID = []
Mapped_by_symbol = 0
Mapped_by_alias = 0
for i, row in Unlinked_AllenMouseGenes.iterrows():
    if row["section_data_set_id"] == 68269058:
        print(row)
    if str(row["gene_acronym"]) != "nan":
        allen_symbol = row["gene_acronym"]
    else:
        allen_symbol = None
    if str(row["gene_alias_tags"]) != "nan":
        allen_alias = row["gene_alias_tags"].split()
    else:
        allen_alias = []
    allen_section_id = row["section_data_set_id"]
    FLAG_FIND = False
    for k in Mouse_MissSectionID:
        symbol = Mouse2Human_Genes[k]["symbol"]
        if symbol.lower() == allen_symbol.lower():
            Mouse2Human_Genes[k]["allen_section_data_set_id"].append(allen_section_id)
            FLAG_FIND = True
            Mapped_by_symbol += 1
            break
        else:
            for alias in allen_alias:
                if alias == "C9":
                    print(row)
                if alias.lower() == symbol.lower():
                    Mouse2Human_Genes[k]["allen_section_data_set_id"].append(allen_section_id)
                    FLAG_FIND = True
                    Mapped_by_alias += 1
                    print(alias)
                    break
    if not FLAG_FIND:
        Symbol_Failed_ID.append(allen_section_id)
print(Mapped_by_symbol, Mapped_by_alias)
print(len(Symbol_Failed_ID))

In [ ]:
Unlinked_AllenMouseGenes = AllenMouseGenes[AllenMouseGenes["section_data_set_id"].isin(
    Symbol_Failed_ID)]

In [ ]:
Unlinked_AllenMouseGenes

In [ ]:
DIR = RootDIR + 'dat/allen-mouse-exp/'
with open(DIR + 'human2mouse.0420.json', 'w') as outfile:
    json.dump(Human2Mouse_Genes, outfile)
with open(DIR + 'mouse2sectionID.0420.json', 'w') as outfile:
    json.dump(Mouse2Human_Genes, outfile)
Unlinked_AllenMouseGenes.to_csv(DIR + "Allen_Unlinked_SectionIDs.0415.csv", index=False)

In [ ]:
Mouse2Human_Genes

In [ ]:
Mouse_MissSectionID = []
for K, V in Mouse2Human_Genes.items():
    if len(V["allen_section_data_set_id"]) == 0:
        Mouse_MissSectionID.append(K)
print(len(Mouse_MissSectionID))

# Expression Level Mat construction

In [ ]:
STR_Meta = pd.read_csv(RootDIR + 'dat/allen-mouse-exp/allen_brain_atlas_structures.csv')
STR_Meta.dropna(inplace=True, subset=["atlas_id"])
STR_Meta["atlas_id"] = STR_Meta["atlas_id"].astype(int)
#STR_Meta["parent_structure_id"] = STR_Meta["parent_structure_id"].astype(int)
STR_Meta = STR_Meta.set_index("atlas_id")
for i, row in STR_Meta.iterrows():
    STR_Meta.loc[i, "Name2"] = modify_str(row["safe_name"])


In [ ]:
STR_Meta[STR_Meta["depth"]==0]

In [ ]:
Selected_STRs = LoadList(RootDIR + 'dat/allen-mouse-exp/Structures.txt')


In [ ]:
STR_Meta_2 = STR_Meta[STR_Meta["Name2"].isin(Selected_STRs)]
STR_Meta_2 = STR_Meta_2.sort_values("Name2")
STR_Meta_2.to_csv(RootDIR + 'dat/allen-mouse-exp/Selected_213_STRs.Meta.csv')


In [ ]:
STR_Meta_2.head(2)

In [ ]:
Count1, Count2, Count3 = 0,0,0
for K, V in Mouse2Human_Genes.items():
    Count1 += 1
    #print(V)
    if len(V["humanHomo"])>0:
        Count2 += 1
    if len(V["allen_section_data_set_id"]) > 0:
        Count3 += 1
print(Count1, Count2, Count3)

In [ ]:
# Check Human Genes
Count1, Count2, Count3 = 0,0,0
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Count1 += 1
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    if len(mouseHomo) > 0:
        Count2 += 1
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"] 
        g_All_Section_ID.extend(section_ids)
    #PRINT(g_All_Section_ID)
    if len(g_All_Section_ID) > 0:
        Count3 += 1
print(Count1, Count2, Count3)

In [ ]:
# Check Exp Level Cross Expriments
DataDIR = RootDIR + 'dat/allen-mouse-exp/allen-mouse-brain-altas-str-unionzed/'
All_Genes = []
All_ExpEnergy = []
Counts = 0

Exp1 = []
Exp2 = []
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"]
        g_All_Section_ID.extend(section_ids)
    if len(g_All_Section_ID) <= 1:
        continue

    g_All_dat = []
    for section in g_All_Section_ID:
        dat_df = pd.read_csv("{}/{}.csv".format(DataDIR, section))
        _ = dat_df[dat_df["structure_id"]==997]
        if len(_) > 0:
            exp_energy = _["expression_energy"].values[0]
            g_All_dat.append(exp_energy)
        else:
            continue
    #print(entrez, Symbol, v["mouseHomo"], g_All_Section_ID)
    #print(g_All_dat)
    if len(g_All_dat)>=2:
        Exp1.append(g_All_dat[0])
        Exp2.append(g_All_dat[1])

    Counts += 1
    #if Counts > 1000:
    #    break


In [ ]:
print(len(Exp1), len(Exp2))

In [ ]:
plt.scatter(Exp1, Exp2, s=1)
plt.xlabel("Section1")
plt.ylabel("Section2")
print(pearsonr(Exp1, Exp2))

In [ ]:
# Get Root Expression
# Check Exp Level Cross Expriments
DataDIR = RootDIR + 'dat/allen-mouse-exp/allen-mouse-brain-altas-str-unionzed/'
All_Genes = []
All_ExpEnergy = []
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"]
        g_All_Section_ID.extend(section_ids)
    g_All_dat = []
    for section in g_All_Section_ID:
        dat_df = pd.read_csv("{}/{}.csv".format(DataDIR, section))
        _ = dat_df[dat_df["structure_id"]==997]
        if len(_) > 0:
            exp_energy = _["expression_energy"].values[0]
            g_All_dat.append(exp_energy)
    g_avg_exp_energy = np.nanmean(g_All_dat)
    All_ExpEnergy.append(g_avg_exp_energy)
All_ExpEnergy = np.array(All_ExpEnergy)    
        

In [ ]:
print(len(All_ExpEnergy), len(Human2Mouse_Genes.keys()))

In [ ]:
df = pd.DataFrame(data={"Genes":list(Human2Mouse_Genes.keys()), "EXP":All_ExpEnergy})
df = df.dropna(subset=["EXP"])
df.shape

In [ ]:
df.to_csv(RootDIR + 'dat/allen-mouse-exp/ExpMatchFeatures.csv', index=False)


In [ ]:
#Human2Mouse_Genes = json.load(open(RootDIR + 'dat/allen-mouse-exp/human2mouse.0420.json', "r"))
#Mouse2Human_Genes = json.load(open(RootDIR + 'dat/allen-mouse-exp/mouse2sectionID.0420.json', "r"))

In [ ]:
DataDIR = RootDIR + 'dat/allen-mouse-exp/allen-mouse-brain-altas-str-unionzed/'
All_Genes = []
All_ExpEnergy = []
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"]
        g_All_Section_ID.extend(section_ids)
    #print(g_All_Section_ID)
    g_All_dat = []
    if len(g_All_Section_ID) <= 0:
        continue
    for section in g_All_Section_ID:
        #print(section)
        dat_df = pd.read_csv("{}/{}.csv".format(DataDIR, section))
        #print(dat_df)
        dat = []
        for str_id, row in STR_Meta_2.iterrows():
            _id = row["id"]
            _ = dat_df[dat_df["structure_id"]==_id]
            if len(_) > 0:
                exp_energy = np.log2(1+_["expression_energy"].values[0])
            else:
                exp_energy = np.nan
                
            dat.append(exp_energy)
        g_All_dat.append(dat)
    g_All_dat = np.array(g_All_dat)
    #print(g_All_dat)
    
    g_avg_exp_energy = np.nanmean(g_All_dat, axis=0)
    #print(g_avg_exp_energy)
    
    All_Genes.append(entrez)
    All_ExpEnergy.append(g_avg_exp_energy)
    
All_ExpEnergy = np.array(All_ExpEnergy)    


In [ ]:
ExpMat = pd.DataFrame(data=All_ExpEnergy, index=All_Genes, columns=STR_Meta_2["Name2"].values)
ExpMat2 = ExpMat.dropna(axis=0, how='all', inplace=False)
print(ExpMat2.shape, ExpMat.shape)

In [ ]:
ExpMat.to_csv(RootDIR + 'dat/allen-mouse-exp/ExpLevel.csv.gz', compression='gzip')


In [ ]:
ExpMat.head(2)

# Expression Zscore

In [ ]:
ExpZscoreMat = ZscoreConverting(ExpMat)
ExpZscoreMat.to_csv(RootDIR + 'dat/allen-mouse-exp/ExpLevel.Zscore.csv.gz', compression='gzip')

In [ ]:
#exp_level = pd.read_csv("../dat/allen-mouse-exp/jon/exp_level-before_norm.csv", 
#                           index_col="ROW")
ExpMat = pd.read_csv("dat/jon_data/expression_energy-human-connectome_struc-log2-qn.csv", 
                           index_col="ROW")
ExpMat = ExpMat.loc[Genes,:]
#ExpMat = pd.read_csv("/Users/jiayao/Work/ASD_Circuits/dat/allen-mouse-exp/jw_exp_mat.0418.csv", 
#                           index_col=0)

In [ ]:
#ExpMat_Log = ExpMat.copy(deep=True)
#for c in ExpMat.columns.values:
#    ExpMat[c] = np.log2(1+ExpMat_Log[c])
#ExpMat_Log_QN = quantileNormalize(ExpMat)
#ExpMat_Log.to_csv("/Users/jiayao/Work/ASD_Circuits/dat/allen-mouse-exp/jw_exp_mat.log2.0418.csv")


In [ ]:
df_z = []
for i in ExpMat.index.values :
    i_z = ZscoreConverting(ExpMat.loc[i,:].values)
    df_z.append(i_z)
ExpZscoreMat = pd.DataFrame(data=df_z, 
                            columns=ExpMat.columns.values, 
                            index=ExpMat.index.values)

ExpZscoreMat.to_csv("/Users/jiayao/Work/ASD_Circuits/dat/allen-mouse-exp/AllenMouse_z1_mat.0511.csv")


In [ ]:
ExpMat_Log_QN = quantileNormalize(ExpMat_Log)
df_z = []
for i in ExpMat_Log_QN.index.values :
    i_z = ZscoreConverting(ExpMat_Log_QN.loc[i,:].values)
    df_z.append(i_z)
ExpZscoreMat2 = pd.DataFrame(data=df_z, 
                            columns=ExpMat_Log_QN.columns.values, 
                            index=ExpMat_Log_QN.index.values)
ExpZscoreMat2.to_csv("/Users/jiayao/Work/ASD_Circuits/dat/allen-mouse-exp/jw_z1_mat.qn.0418.csv")




### Combine Z2 Matrix

In [ ]:
Z2_split_dir = "/Users/jiayao/Work/ASD_Circuits/dat/allen-mouse-exp/Mouse.Mean.UniformKernel.Z2scores/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/AllenMouseBrain_Z2bias.csv")

In [ ]:
Z2_split_dir = "../dat/AllenMouse.BrainSpanMatch.Z2scores/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("../dat/allen-mouse-exp/AllenMouseBrain_Z2bias.brainspanMatch.csv")

In [ ]:
JC_Z2 = pd.read_csv("dat/Jon_data/exp_bias.csv", index_col=0)
dat1, dat2 = [], []
for g in ASD_Genes:
    for str_ in JC_Z2.columns.values:
        x1 = Z2_MAt.loc[g, str_]
        x2 = JC_Z2.loc[g, str_]
        if x1 == x1 and x2 == x2:
            dat1.append(x1)
            dat2.append(x2)
plt.scatter(dat1, dat2, s=0.1)
plt.xlabel("JW")
plt.ylabel("JC")
print(spearmanr(dat1, dat2, nan_policy="omit"))
print(pearsonr(dat1, dat2))

In [ ]:
# Z2 Mat with neuron norm

Z2_split_dir = "../dat/AllenMouse.NeuroNorm.Z2score/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("../dat/allen-mouse-exp/AllenMouseBrain_Z2bias.neuronorm.csv")